<a href="https://colab.research.google.com/github/Lokesh1854/Short-Term-Load-Forecasting/blob/main/parrallel_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, concatenate, Dense

# Define input shape
input_shape = (48,1)  # Assuming timesteps and features are predefined

# Define the first LSTM branch
input_layer = Input(shape=(48,1))
lstm_1 = LSTM(100)(input_layer)
lstm_2 = Dense(64, activation = 'relu')(lstm_1)
lstm_3 = Dense(32, activation = 'relu')(lstm_2)
lstm_4 = Dense(16, activation = 'relu')(lstm_3)
lstm_5 = Dense(1, activation = 'relu')(lstm_4)
# Define the second LSTM branch
lstm_11 = LSTM(100)(input_layer)
lstm_21 = Dense(64, activation = 'relu')(lstm_11)
lstm_31 = Dense(32, activation = 'relu')(lstm_21)
lstm_41 = Dense(16, activation = 'relu')(lstm_31)
lstm_51 = Dense(1, activation = 'relu')(lstm_41)
# Concatenate the outputs of both LSTM branches
merged = concatenate([lstm_5, lstm_51])

# Add further layers if needed
# Example:
output_layer = Dense(1, activation='relu')(merged)


# Create the model
model1 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model1.compile(optimizer='adam', loss='mape')

# Print model summary
model1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 48, 1)]              0         []                            
                                                                                                  
 lstm (LSTM)                 (None, 100)                  40800     ['input_1[0][0]']             
                                                                                                  
 lstm_1 (LSTM)               (None, 100)                  40800     ['input_1[0][0]']             
                                                                                                  
 dense (Dense)               (None, 64)                   6464      ['lstm[0][0]']                
                                                                                              

In [ ]:
# creating time_steps of 1 day -- 24 x 6
look_back = 2*24

def prepare_lstm_data(data,look_back):
  X_lstm,Y_lstm = [],[]
  for i in range(len(data)-look_back):
    X_lstm.append(np.array(data.iloc[i:i+look_back,-1]))
    Y_lstm.append(np.array(data.iloc[i+look_back,-1]))
  return np.array(X_lstm),np.array(Y_lstm)


In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
cluster_path = "/content/drive/MyDrive/STLF/DataSet/Cluster 0"

mape_err = []

for csvFile in os.listdir(cluster_path):
  csvPath = os.path.join(cluster_path,csvFile)
  df = pd.read_csv(csvPath)

  X_lstm,Y_lstm = prepare_lstm_data(df,look_back)
  X_lstm_train, X_lstm_test, Y_lstm_train, Y_lstm_test = train_test_split(X_lstm, Y_lstm, test_size=0.1)
  model1.fit(X_lstm_train, Y_lstm_train, epochs=25, validation_split=0.1, batch_size=32)

  mape_err.append([csvFile,model1.evaluate(X_lstm_test,Y_lstm_test)])

Epoch 1/25
531/531 [==============================] - 44s 72ms/step - loss: 13.4451 - val_loss: 10.0494
Epoch 2/25
531/531 [==============================] - 32s 60ms/step - loss: 9.9391 - val_loss: 9.8329
Epoch 3/25
531/531 [==============================] - 32s 61ms/step - loss: 9.4261 - val_loss: 8.9596
Epoch 4/25
531/531 [==============================] - 33s 62ms/step - loss: 9.2998 - val_loss: 9.3876
Epoch 5/25
531/531 [==============================] - 36s 67ms/step - loss: 9.1356 - val_loss: 8.8517
Epoch 6/25
531/531 [==============================] - 35s 67ms/step - loss: 9.1108 - val_loss: 8.7941
Epoch 7/25
531/531 [==============================] - 33s 63ms/step - loss: 9.0978 - val_loss: 8.9008
Epoch 8/25
531/531 [==============================] - 32s 61ms/step - loss: 9.0347 - val_loss: 8.7601
Epoch 9/25
531/531 [==============================] - 32s 61ms/step - loss: 8.9812 - val_loss: 9.2324
Epoch 10/25
531/531 [==============================] - 32s 61ms/step - loss: 8.9

In [ ]:
mape_err

[['block_16.csv', 8.47779369354248],
 ['block_27.csv', 10.478328704833984],
 ['block_4.csv', 8.067485809326172],
 ['block_9.csv', 8.116782188415527]]

In [ ]:
os.chdir(cluster_path)

In [ ]:
model1.save('lstm_parallel')